**Cat vs Dog classification** done using pretrained models: AlexNet, ResNet, VGG16, Inception-v3, MobileNet

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:48<00:00, 24.5MB/s]
100% 1.06G/1.06G [00:48<00:00, 23.8MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet

In [ ]:
# Define input image dimensions
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)

In [ ]:
# AlexNet model

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

model = Sequential()

model.add(Conv2D(96,kernel_size=(11,11), strides=(4,4),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),strides=2,padding='valid'))

model.add(Conv2D(256,kernel_size=(5,5),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),strides=2,padding='valid'))

model.add(Conv2D(384,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(384,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.summary()

In [ ]:
# Load the pre-trained models
models = {
    "AlexNet": model(),
    "ResNet": ResNet50(weights='imagenet', include_top=False, input_shape=input_shape),
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=input_shape),
    "Inception-v3": InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape),
    "MobileNet": MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
}

87910968/87910968 [==============================] - 5s 0us/step


17225924/17225924 [==============================] - 2s 0us/step


In [ ]:
# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Define and compile models
results = {}

for model_name, model in models.items():
    model.layers.pop()  # Remove the last layer
    for layer in model.layers:
        layer.trainable = False  # Freeze the pre-trained layers
    x = tf.keras.layers.Flatten()(model.output)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    cnn_model = tf.keras.Model(model.input, x)
    cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = cnn_model.fit(train_generator, epochs=10, validation_data=validation_generator)
    results[model_name] = history.history['val_accuracy']

Epoch 1/5
625/625 [==============================] - 112s 166ms/step - loss: 0.8607 - accuracy: 0.6190 - val_loss: 0.5928 - val_accuracy: 0.6988
Epoch 2/5
625/625 [==============================] - 103s 164ms/step - loss: 0.7238 - accuracy: 0.6712 - val_loss: 0.6343 - val_accuracy: 0.6904
Epoch 3/5
625/625 [==============================] - 103s 164ms/step - loss: 0.7085 - accuracy: 0.6855 - val_loss: 0.7329 - val_accuracy: 0.6384
Epoch 4/5
625/625 [==============================] - 104s 167ms/step - loss: 0.6696 - accuracy: 0.7020 - val_loss: 0.7052 - val_accuracy: 0.6724
Epoch 5/5
625/625 [==============================] - 103s 164ms/step - loss: 0.6191 - accuracy: 0.7199 - val_loss: 0.7375 - val_accuracy: 0.6688
Epoch 1/5
625/625 [==============================] - 146s 221ms/step - loss: 0.2285 - accuracy: 0.9042 - val_loss: 0.2293 - val_accuracy: 0.8994
Epoch 2/5
625/625 [==============================] - 128s 205ms/step - loss: 0.1335 - accuracy: 0.9466 - val_loss: 0.1750 - val_ac

In [ ]:
# Display accuracy of each model
for model_name, accuracy in results.items():
    print(f"{model_name}: Validation Accuracy - {np.max(accuracy)}")